In [1]:
from fraudfunc import fdr_score, fdr_summary, randomized_search, grid_search
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
mod = pd.read_csv('modeling_data.csv')
oot = pd.read_csv('oot_data.csv')

In [3]:
print('mod:',mod.shape)
print('oot:',oot.shape)

mod: (80632, 27)
oot: (12427, 27)


In [4]:
X_mod=mod.iloc[:,:-1]
y_mod=mod.iloc[:,-1]
X_oot=oot.iloc[:,:-1]
y_oot=oot.iloc[:,-1]

In [5]:
print('mod fraud:',sum(y_mod))
print('oot fraud:',sum(y_oot))

mod fraud: 868
oot fraud: 179


In [6]:
lr = LogisticRegression()
lr.fit(X_mod,y_mod)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
y_mod_pred_prob=lr.predict_proba(X_mod)[:,1]
y_oot_pred_prob=lr.predict_proba(X_oot)[:,1]
print(fdr_score(y_mod_pred_prob,y_mod))
print(fdr_score(y_oot_pred_prob,y_oot))

0.7004608294930875
0.441340782122905


In [8]:
rf=RandomForestClassifier(n_estimators=10)
summary=fdr_summary(rf, X_mod, y_mod, X_oot, y_oot)
summary

,trn,tst,oot
1,1.000000,0.896154,0.586592
2,0.998355,0.896154,0.569832
3,1.000000,0.903846,0.670391
4,1.000000,0.919231,0.636872
5,0.998355,0.919231,0.636872
6,1.000000,0.853846,0.603352
7,1.000000,0.915385,0.620112
8,0.998355,0.915385,0.541899
9,1.000000,0.911538,0.575419
10,1.000000,0.884615,0.636872


In [18]:
from sklearn.utils import resample
import numpy as np

In [9]:
test_grid = {
    'n_estimators': [5, 10],
    'criterion': ["gini", "entropy"],
    'max_depth':[10, 20, None],
    'min_samples_split': [500, 1000, 1500],
    #'min_samples_leaf': [30, 50, 100],
    #'max_features': [5, 10, 25],
    #'n_jobs': [-1]
}

In [13]:
rf=RandomForestClassifier()
randomized_search(rf, test_grid, X_mod, y_mod, n_iter=10, n_jobs=-1)

Total combination: 36
Randomized search size: 10
{'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 1000, 'n_estimators': 10}
{'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 1500, 'n_estimators': 5}
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 500, 'n_estimators': 10}
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 1000, 'n_estimators': 5}
{'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 1000, 'n_estimators': 5}
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 500, 'n_estimators': 5}
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 1000, 'n_estimators': 10}
{'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 500, 'n_estimators': 5}
{'criterion': 'entropy', 'max_depth': 20, 'min_samples_split': 1500, 'n_estimators': 5}, FDR: 0.64%, 2 sec elapsed
{'criterion': 'gini', 'max_depth': None, 'min_samples_split': 1000, 'n_estimators': 10}
{'criterion': 'gini', 'max_depth': 10, 'min_sampl

(0.6865384615384615,
 {'criterion': 'entropy',
  'max_depth': 20,
  'min_samples_split': 500,
  'n_estimators': 10})

In [14]:
rf=RandomForestClassifier()
grid_search(rf, test_grid, X_mod, y_mod, n_jobs=-1,verbose=False)

Total combination: 36
Total time elapsed: 0.3 minutes


(0.7176923076923077,
 {'criterion': 'gini',
  'max_depth': 20,
  'min_samples_split': 500,
  'n_estimators': 10})